In [1]:
import re, string, os
import pandas as pd
import numpy as np
import jiwer
from jiwer import wer
import pickle
import regex as re
import nltk
import itertools
from sklearn.ensemble import RandomForestClassifier
#nltk.download('words')
import warnings
warnings.filterwarnings("ignore")

from functions import *

# Importing data

In [2]:
train = pd.read_pickle("./Data/alignData/ent_with_ref.pkl") 
test = pd.read_pickle("./Data/alignment_test.pkl") 
voting = pd.read_pickle("./Data/assessment_voting.pkl") 

reference_texts_validation = pickle.load(open('./Data/reference_texts_validation.pkl', 'rb'))
reference_texts_test = pickle.load(open('./Data/reference_texts_test.pkl', 'rb'))
reference_texts_train = pickle.load(open('./Data/reference_texts_train.pkl', 'rb'))

# Encoding words

In [3]:
le, col = word_encoding(train, test)
train[col] = train[col].apply(lambda x :le.transform(x))

# Training data preparation

In [4]:
train_data, col_name, machines, match = train_data_v1(train, reference_texts_train, reference_texts_validation)

In [5]:
# Upsampling
train_data = upsampling(train_data)
train_data.to_pickle("./Data/train_data_upsampled.pkl")

100%|██████████| 15047/15047 [07:26<00:00, 33.66it/s]


In [6]:
# machine tracker creation
train_data_ref = train_data.copy()
train_data = machine_tracker(train_data, col_name, machines, match)
train_data.to_pickle("./Data/train_data_final.pkl")

100%|██████████| 371207/371207 [28:59<00:00, 213.36it/s]


# Random forest

In [9]:
train_data = pd.read_pickle("./Data/train_data_final.pkl")

In [8]:
X_train, y_train = train_test_split(train_data)
regr = RandomForestClassifier(n_estimators = 20, min_samples_leaf=3, n_jobs=-1)
regr.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [6]:
#pickle.dump(regr, open('random_forest_classifier.sav', 'wb'), protocol=4)

# Test data

In [9]:
test_reduced, col_test, reference_texts_test_reduced = test_data_preparation_1(test, reference_texts_test)
test_reduced[col_test] = test_reduced[col_test].apply(lambda x :le.transform(x))
test_reduced_df, machines_test, match_test = test_data_preparation_2(test_reduced)
test_reduced_df = first_prediction(test_reduced_df, regr)

In [ ]:
test_reduced_df2 = test_predictions(test_reduced_df, machines_test, match_test, regr, col_test)
#test_reduced_df2.to_pickle("./Test_predictions_final(full)_pickl.pkl")

# Comparison Analysis

In [10]:
ori_df = pd.read_pickle("./Data/Dataframe/df_clean.pkl") 
ori_df = raw_data_preparation(ori_df)

test_reduced_df = pd.read_pickle("./Data/Test_predictions_final(full)_pickl.pkl")

assessment_model = assessment_dataframe_creation(ori_df, test_reduced_df, reference_texts_test_reduced, le)
assessment_model.to_pickle("./Data/assessment_model.pkl")

100%|██████████| 2802/2802 [01:08<00:00, 41.09it/s]


In [12]:
#pd.set_option('display.max_colwidth',-1)
#assessment_model[assessment_model['model_wer'] < assessment_model['machine_wer']]

## Example

In [14]:
sent = 65
comparison(sent, test_reduced_df, reference_texts_test_reduced, le, assessment_model, ori_df)

Ground truth: 
as we start to have cyborg bodies for exploration or for other means in in many sort of post human futures that i that i can imagine as a science fiction fan so having done these these

Model Prediction:
as we start to have desegregation bodies for exploration of for other means and in many sort of post human that that i can imagine as a science fiction fan so having done these these um

Best machine : 4_iaebglebg
as we start to have cyborg bodies for exploration for other means and in many sort of post human future is that that i can imagine as a science fiction fan so having done these these

-----------------------------------------------------------------------------
WER model:         0.16216216216216217
WER best machine:  0.13513513513513514
-----------------------------------------------------------------------------
Mean WER model:    0.22380283015956312
Mean WER model:    0.09504199017870853
-----------------------------------------------------------------------